<a href="https://colab.research.google.com/github/okweipeng/Neural-Machine-Translation/blob/main/CST4812_Final_Project_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###**CST4812 Project Topic: Neural Machine Translation**

**Use of the imports in Pytorch and Transformers Libraries**

**Note: Please Use/Connect To GPU within Google Colab to run all fields. As CPU will take alot longer!**

In [ ]:
#Install the transformers, sentencepiece, and datasets libraries for NLP tasks
!pip install transformers sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [ ]:
# Importing the neccessary libraies to perform the tasks for Neural Machine Translation
from datasets import load_dataset
import numpy as np
# Use of the imports of pytorch & transformers for this project
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

In [ ]:
# Setting the pre-trained model
# Specifies the repository on Hugging Face's model hub to use the 'mt5-small' model (smallest model possible)
model_repo = 'google/mt5-small'

# Define the maximum sequence length for processing text inputs
max_seq_len = 20

In [ ]:
# Load a tokenizer from a pre-trained model specified by 'model_repo' for text tokenization
tokenizer = AutoTokenizer.from_pretrained(model_repo)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Load a pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
# Move the model to GPU to enable faster computation
model = model.cuda()

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Define a string that will be input to a tokenizer
example_input_str = '<jp>This is a test.'
# Tokenize the input string and convert it to tensor format, suitable for model input
input_ids = tokenizer.encode(example_input_str, return_tensors='pt')
print('Input IDs:', input_ids)

# Convert the list of input IDs back into readable tokens to verify the tokenization process
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# Print the tokens corresponding to the input IDs
print('Tokens:', tokens)

Input IDs: tensor([[ 1042,  3889,   669, 13673,   339,   259,   262,  2978,   260,     1]])
Tokens: ['▁<', 'jp', '>', 'This', '▁is', '▁', 'a', '▁test', '.', '</s>']


In [ ]:
# sorted(tokenizer.vocab.items(), key=lambda x: x[1])

In [ ]:
# Load the ALT dataset using the Hugging Face `datasets` library
dataset = load_dataset('alt')

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [ ]:
# Extract the training subset from the dataset dictionary
train_dataset = dataset['train']
# Extract the testing subset from the dataset dictionary
test_dataset = dataset['test']

In [ ]:
# Language that can be used for the first dataset
train_dataset[0]

{'SNT.URLID': '80188',
 'SNT.URLID.SNTID': '1',
 'url': 'http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal',
 'translation': {'bg': 'ফ্রান্সের প্যারিসের পার্ক দি প্রিন্সেস-এ হওয়া ২০০৭-এর রাগবি বিশ্বকাপের পুল সি-তে ইটালি পর্তুগালকে ৩১-৫ গোলে হারিয়েছে।',
  'en': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes, Paris, France.',
  'en_tok': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes , Paris , France .',
  'fil': 'Natalo ng Italya ang Portugal sa puntos na 31-5 sa Grupong C noong 2007 sa Pandaigdigang laro ng Ragbi sa Parc des Princes, Paris, France.',
  'hi': '2007 में फ़्रांस, पेरिस के पार्क डेस प्रिंसेस में हुए रग्बी विश्व कप के पूल C में इटली ने पुर्तगाल को 31-5 से हराया।',
  'id': 'Italia berhasil mengalahkan Portugal 31-5 di grup C dalam Piala Dunia Rugby 2007 di Parc des Princes, Paris, Perancis.',
  'ja': 'フランスのパリ、パルク・デ・プランスで行われた2007年ラグビーワールドカップのプールCで、イタリアは31対5でポルトガルを

In [ ]:
LANG_TOKEN_MAPPING = {
    'en': '<en>', # maps to lang: english
    'hi': '<hi>', # maps to lang: hindi
    'ja': '<ja>', # maps to lang: japanese
    'zh': '<zh>'  # maps to lang: chinese
}

In [ ]:
# Define a dictionary of special tokens
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
# Add these special tokens to the tokenizer
tokenizer.add_special_tokens(special_tokens_dict)
# Resizing the model's token embeddings
model.resize_token_embeddings(len(tokenizer))

Embedding(250104, 512)

In [ ]:
# Encode the input string into token IDs suitable for input to a transformer model
token_ids = tokenizer.encode(
    example_input_str,  # The input text string to be encoded
    return_tensors='pt',  # Return the result as a PyTorch tensor
    padding='max_length', # Pad the sequence to a maximum length
    truncation=True,  # Truncate the sequence to fit the maximum length
    max_length=max_seq_len) # Specify the maximum length for the sequence

# Print the encoded token IDs
print(token_ids)

tensor([[ 1042,  3889,   669, 13673,   339,   259,   262,  2978,   260,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])


In [ ]:
def encode_input_str(text, target_lang, tokenizer, seq_len, lang_token_map=LANG_TOKEN_MAPPING):
# Get the language token for the target language
  target_lang_token = lang_token_map[target_lang]

 # Encode the text with the target language token, tokenize and pad/truncate it to seq_len
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

def encode_target_str(text, tokenizer, seq_len, lang_token_map=LANG_TOKEN_MAPPING):
# Encode the target text, tokenize and pad/truncate it to seq_len
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]

def format_translation_data(translations, lang_token_map, tokenizer, seq_len=128):
# Randomly select two languages from the available languages in lang_token_map
  langs = list(lang_token_map.keys())
  input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

 # Fetch the translation texts for the randomly selected languages
  input_text = translations[input_lang]
  target_text = translations[target_lang]

# Return None if either text is missing
  if input_text is None or target_text is None:
    return None
# Encode the input and target text
  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map
  )

  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map
  )
# Return the token IDs for both input and target texts
  return input_token_ids, target_token_ids


def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []
  # Process each translation set in the batch
  for translation_set in batch['translation']:
    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_seq_len)

# if no formatted data is returned continue
    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
# Append the input and target IDs, adding an extra dimension
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

 # Concatenate all input and target IDs and move them to GPU for processing
  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
# Shuffle the dataset
  dataset = dataset.shuffle()
# Yield batches of data
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
# Transform each raw batch and yield the result
    yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [ ]:
# Format the first translation example from the training dataset
in_ids, out_ids = format_translation_data(train_dataset[0]['translation'], LANG_TOKEN_MAPPING, tokenizer)

# Convert the token IDs for the input language back to tokens (words/subwords) and print them
print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Initialize a data generator for the training dataset
data_gen = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, 8)
# Retrieve the first batch of data from the generator.
data_batch = next(data_gen)

# Print the shape of the input tensor in the first batch
print('Input shape:', data_batch[0].shape)
# Print the shape of the output tensor in the first batch
print('Output shape:', data_batch[1].shape)

<hi> ▁ フランス の パリ 、 パル ク ・ デ ・ プラン ス で 行われた 2007 年 ラグビー ワールド カップ の プール C で 、 イタリア は 31 対 5 で ポル ト ガル を下 した 。 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁2007 ▁में ▁फ़ ्रा ंस , ▁पे रिस ▁के ▁पार् क ▁डे स ▁ प्रि ंस ेस ▁में ▁हु ए ▁ र ग् बी ▁विश्व ▁कप ▁के ▁पू ल ▁C ▁में ▁ इट ली ▁ने ▁पुर् त गाल ▁को ▁3 1-5 ▁से ▁हर ाया । </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

In [ ]:
# Set the number of training epochs of the number through the training dataset
n_epochs = 5
# Set the size of the training batches, which dictates how many examples are processed together in one forward/backward pass
batch_size = 16
# Set the frequency of printing the training progress
print_freq = 50
# Set the learning rate for the optimizer
lr = 5e-4
# Calculate the total number of batches needed to complete one epoch
n_batches = int(np.ceil(len(train_dataset)/ batch_size))
# Calculate the total number of steps
total_steps = n_epochs * n_batches
# Set the number of warm-up steps as 1% of the total steps
n_warmup_steps = int(total_steps * 0.01)

In [ ]:
# Initialize the AdamW optimizer with the model parameters and a specified learning rate
optimizer = AdamW(model.parameters(), lr=lr)

# Setting up a linear learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Initialize an empty list to store the loss values during training
losses = []

In [ ]:
def eval_model(model, gdataset, max_iters=8):
  # Initialize the data generator for the given dataset using the global language token map and tokenizer.
  test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)

  eval_losses = []  # List to store loss values for each iteration

  # Iterate through the test generator to process batches for evaluation.
  for i, (input_batch, label_batch) in enumerate (test_generator):
    if i >= max_iters:
      break

# Forward pass through the model to compute outputs and loss for the current batch.
    model_out = model.foward(
        input_ids = input_batch,
        labels = label_batch)
    # Append the computed loss
    eval_losses.append(model_out.loss.item())

# Compute and return the mean of the evaluation losses collected
  return np.mean(eval_losses)

**Note: Does not need to be fully ran, as it goes through the learning rate for precised translations from one text to another**

In [ ]:
# Loop over each epoch
for epoch_idx in range(n_epochs):

  # Create a new data generator for each epoch
  data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)
  # Iterate over each batch in the dataset
  for batch_idx, (input_batch, label_batch) \
      in  tqdm_notebook(enumerate(data_generator), total=n_batches):

      optimizer.zero_grad()

# Forward pass: compute the model's predictions for the current input batch
      model_out = model.forward(
          input_ids = input_batch,
          labels = label_batch
      )
# Compute loss: difference between the model's predictions and the true labels
      loss = model_out.loss
      losses.append(loss.item()) # Store the loss for this batch
      loss.backward()   # Backward pass: compute the gradient of the loss with respect to model parameters
      optimizer.step()  # Update model parameters
      scheduler.step()  # Update the learning rate

# Print average loss and learning rate at intervals specified by print_freq
      if (batch_idx + 1) % print_freq == 0:
            avg_loss = np.mean(losses[-print_freq:])  # Calculate average loss
            print('Epoch: {} | Step: {} | Avg loss: {:.3f} | lr: {}'.format(
                epoch_idx + 1, batch_idx + 1, avg_loss, scheduler.get_last_lr()[0]))

<ipython-input-19-0ba9a599b222>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  in  tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 1 | Step: 50 | Avg loss: 13.270 | lr: 0.00044642857142857147
Epoch: 1 | Step: 100 | Avg loss: 6.237 | lr: 0.0004960707269155207
Epoch: 1 | Step: 150 | Avg loss: 5.036 | lr: 0.0004916056438649759
Epoch: 1 | Step: 200 | Avg loss: 4.519 | lr: 0.00048714056081443114
Epoch: 1 | Step: 250 | Avg loss: 4.282 | lr: 0.0004826754777638864
Epoch: 1 | Step: 300 | Avg loss: 4.115 | lr: 0.0004782103947133417
Epoch: 1 | Step: 350 | Avg loss: 3.978 | lr: 0.00047374531166279694
Epoch: 1 | Step: 400 | Avg loss: 3.913 | lr: 0.0004692802286122522
Epoch: 1 | Step: 450 | Avg loss: 3.776 | lr: 0.0004648151455617075
Epoch: 1 | Step: 500 | Avg loss: 3.703 | lr: 0.00046035006251116275
Epoch: 1 | Step: 550 | Avg loss: 3.655 | lr: 0.00045588497946061796
Epoch: 1 | Step: 600 | Avg loss: 3.622 | lr: 0.00045141989641007323
Epoch: 1 | Step: 650 | Avg loss: 3.613 | lr: 0.0004469548133595285
Epoch: 1 | Step: 700 | Avg loss: 3.569 | lr: 0.00044248973030898376
Epoch: 1 | Step: 750 | Avg loss: 3.527 | lr: 0.00043802

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 2 | Step: 50 | Avg loss: 3.257 | lr: 0.00039953563136274335
Epoch: 2 | Step: 100 | Avg loss: 3.241 | lr: 0.0003950705483121986
Epoch: 2 | Step: 150 | Avg loss: 3.228 | lr: 0.00039060546526165383
Epoch: 2 | Step: 200 | Avg loss: 3.156 | lr: 0.00038614038221110916
Epoch: 2 | Step: 250 | Avg loss: 3.101 | lr: 0.00038167529916056437
Epoch: 2 | Step: 300 | Avg loss: 3.173 | lr: 0.00037721021611001964
Epoch: 2 | Step: 350 | Avg loss: 3.137 | lr: 0.00037274513305947496
Epoch: 2 | Step: 400 | Avg loss: 3.162 | lr: 0.0003682800500089302
Epoch: 2 | Step: 450 | Avg loss: 3.130 | lr: 0.00036381496695838544
Epoch: 2 | Step: 500 | Avg loss: 3.093 | lr: 0.00035934988390784066
Epoch: 2 | Step: 550 | Avg loss: 3.186 | lr: 0.000354884800857296
Epoch: 2 | Step: 600 | Avg loss: 3.205 | lr: 0.0003504197178067512
Epoch: 2 | Step: 650 | Avg loss: 3.066 | lr: 0.00034595463475620646
Epoch: 2 | Step: 700 | Avg loss: 3.098 | lr: 0.00034148955170566173
Epoch: 2 | Step: 750 | Avg loss: 3.067 | lr: 0.0003370

  0%|          | 0/1131 [00:00<?, ?it/s]

Epoch: 3 | Step: 50 | Avg loss: 3.013 | lr: 0.0002985354527594213
Epoch: 3 | Step: 100 | Avg loss: 2.956 | lr: 0.0002940703697088766
Epoch: 3 | Step: 150 | Avg loss: 2.952 | lr: 0.00028960528665833185
Epoch: 3 | Step: 200 | Avg loss: 2.969 | lr: 0.00028514020360778707
Epoch: 3 | Step: 250 | Avg loss: 2.935 | lr: 0.0002806751205572424
Epoch: 3 | Step: 300 | Avg loss: 2.905 | lr: 0.00027621003750669766
Epoch: 3 | Step: 350 | Avg loss: 2.968 | lr: 0.00027174495445615287
Epoch: 3 | Step: 400 | Avg loss: 2.939 | lr: 0.00026727987140560814
Epoch: 3 | Step: 450 | Avg loss: 3.013 | lr: 0.0002628147883550634
Epoch: 3 | Step: 500 | Avg loss: 2.900 | lr: 0.0002583497053045187
Epoch: 3 | Step: 550 | Avg loss: 2.912 | lr: 0.0002538846222539739
Epoch: 3 | Step: 600 | Avg loss: 2.956 | lr: 0.0002494195392034292
Epoch: 3 | Step: 650 | Avg loss: 2.896 | lr: 0.0002449544561528844
Epoch: 3 | Step: 700 | Avg loss: 2.929 | lr: 0.0002404893731023397
Epoch: 3 | Step: 750 | Avg loss: 2.890 | lr: 0.00023602429

KeyboardInterrupt: 

**Again, does not need to be fully ran, best at epcoch 2 would generate a great model learning/results**

# **Testing Part**

**Original ~ Japanese Text:**

In [ ]:
# Retrieve a Japanese sentence from the first entry of the test dataset
test_sentence = test_dataset[0]['translation']['ja']
print('Raw Input Text:', test_sentence)  # Display the original Japanese text

# Encode the test sentence to input IDs for model processing
input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'hi',
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

# Convert token IDs back to string and print the truncated input text
print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

Raw Input Text: シドニーのランドウィック競馬場の8頭のサラブレッド競走馬が馬インフルエンザに感染していることが確認された。
Truncated input text: <hi> シドニーのランドウィック競馬場の8頭のサラブレッド競走馬</s>


**Translation to Hindi:** **(Japanese to Hindi)**

In [ ]:
# Generate translations from the model using beam search
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=1)
# Loop through each set of generated tokens in the output
for token_set in output_tokens:
# Decode the tokens to a readable string and prints it
    print(tokenizer.decode(token_set, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


संयुक्त राज्य अमेरिका के लंदन काउंटी के 8 राउंड


**Original ~ Hindi Text:**

In [ ]:
# Retrieve a Japanese sentence from the first entry of the test dataset
test_sentence = test_dataset[0]['translation']['hi']
print('Raw Input Text:', test_sentence)  # Display the original Japanese text

# Encode the test sentence to input IDs for model processing
input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'zh',
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

# Convert token IDs back to string and print the truncated input text
print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

Raw Input Text: यह पुष्टि की गई है कि सिडनी में रैंडविक रेसकोर्स के आठ घनी नस्ल के घोड़े इक्वाइन इन्फ्लूएंजा से संक्रमित हैं।
Truncated input text: <zh> यह पुष्टि की गई है कि सिडनी में रैंडविक रे</s>


**Translation of the text to Chinese:** **(Hindi to Chinese)**

In [ ]:
# Generate translations from the model using beam search
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=1)
# Loop through each set of generated tokens in the output
for token_set in output_tokens:
# Decode the tokens to a readable string and prints it
    print(tokenizer.decode(token_set, skip_special_tokens=True))

初步估计,在加利福尼亚州的罗马尼亚州的罗马尼亚


**Original ~ Thai Text**

In [ ]:
# Retrieve a Japanese sentence from the first entry of the test dataset
test_sentence = test_dataset[0]['translation']['th']
print('Raw Input Text:', test_sentence)  # Display the original Japanese text

# Encode the test sentence to input IDs for model processing
input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'en',
    tokenizer = tokenizer,
    seq_len = model.config.max_length,
    lang_token_map = LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cuda()

# Convert token IDs back to string and print the truncated input text
print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

Raw Input Text: ได้เป็นที่ยืนยันแน่นอนแล้วว่าม้าแข่งพันธุ์ดีแปดตัวที่สนามแข่งแรนด์วิค ในเมืงซิดนีย์ได้ติดเชื้อไข้หวัดใหญ่ม้า
Truncated input text: <en> ได้เป็นที่ยืนยันแน่นอนแล้วว่าม้าแข่งพันธุ์ดีแปดตัวที่สนามแข่งแรนด์วิ</s>


**Translation to English:** **(Thai to English)**

In [ ]:
# Generate translations from the model using beam search
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=1)
# Loop through each set of generated tokens in the output
for token_set in output_tokens:
# Decode the tokens to a readable string and prints it
    print(tokenizer.decode(token_set, skip_special_tokens=True))

They are expected to be announced that the winner of the race's 


### ***P.s. Model for Neural Machine Translation seemed to have work in translating text to another.***

***(made a few test tries and it has worked but not as accurate in translation but works....)***